In [1]:
from others import helpers_functional as ff
from others import helpers_layer as module
from others.nets import unet 
from others.Config import Config
from others.dataset import Dataset
import torch
from torch.utils.data.dataloader import DataLoader
import time 

In [2]:
train_noisy_input, train_noisy_target = torch.load(Config.train_data_path)

In [44]:
in_channels=3
out_channels=3
depthChannels_1 = 12
depthChannels_2 = 9

In [40]:
## 15 epoch => 21.6 pnsr
modules = [
            module.Conv2d(in_channels,depthChannels_1,3,stride = 2, padding = 2),
            module.ReLU(),
            module.Conv2d(depthChannels_1,depthChannels_1,3,stride = 2, padding = 2),
            module.ReLU(),
            module.Upsampling(depthChannels_1,depthChannels_2,2,4),
            module.ReLU(),
            module.Upsampling(depthChannels_2,out_channels,2,3),
            module.Sigmoid()
        ]
Net2 = module.Sequential(modules)

In [45]:
modules = [
            module.Conv2d(in_channels,depthChannels_1,2,stride = 2, padding = 0),
            module.ReLU(),
            module.Conv2d(depthChannels_1,depthChannels_2,2,stride = 2, padding = 1),
            module.ReLU(),
            module.Upsampling(depthChannels_2,depthChannels_1,2,2,padding=0),
            module.ReLU(),
            module.Upsampling(depthChannels_1,out_channels,2,3),
            module.Sigmoid()
        ]
Net4 = module.Sequential(modules)

In [34]:
modules = [
            module.Conv2d(in_channels,depthChannels_1,4,stride = 2, padding = 2),
            module.ReLU(),
            module.Conv2d(depthChannels_1,depthChannels_2,4,stride = 2, padding = 2),
            module.ReLU(),
            module.Upsampling(depthChannels_2,depthChannels_1,2,4,padding=1),
            module.ReLU(),
            module.Upsampling(depthChannels_1,out_channels,2,3),
            module.Sigmoid()
        ]
Net1 = module.Sequential(modules)

In [35]:
in_channels=3
out_channels=3
depthChannels_1 = 12
depthChannels_2 = 6

modules = [
            module.Conv2d(in_channels,depthChannels_1,2,stride = 2, padding = 0),
            module.ReLU(),
            module.Conv2d(depthChannels_1,depthChannels_2,2,stride = 2, padding = 1),
            module.ReLU(),
            module.Upsampling(depthChannels_2,depthChannels_1,3,4,padding=0,stride = 2),
            module.ReLU(),
            module.Upsampling(depthChannels_1,out_channels,3,5,padding = 0),
            module.Sigmoid()
        ]
Net4 = module.Sequential(modules)

In [46]:
optimizer = module.Adam(Net4.param(), 0.001, (0.9,0.999), eps = 1e-08)

In [57]:
optimizer.update_lr(0.5,"scale")

In [58]:
optimizer.iter = 10
#optimizer.lr *= 4
optimizer.lr

0.001

In [61]:
def train(train_input, train_target, num_epochs) -> None:
        training_loader = DataLoader(Dataset(train_input, train_target, Config), batch_size=400, shuffle=True,)
        val_noisy_x, val_clean_y = torch.load(Config.val_data_path)
        validation_loader = DataLoader(Dataset(val_noisy_x, val_clean_y, Config), batch_size=Config.batch_size, shuffle=False,)
        epochs_losses = []
        validation_losses = []
        validation_psnr = []
        min_loss = 9999
        
        
        #optimizer = module.Adam(Net4.param(), 0.001, (0.9,0.99), eps = 1e-08)
        #optimizer = module.SGD(Net4.param(), lr = 0.001, mu = 0.9, tau = 0.9)
        criterion = module.MSE()
        
        
        if Config.verbose:
            print("Training started.")

        start = time.time() 
        iters = 1
        for epoch in range(num_epochs):
            # Training
            epoch_loss = 0.0
            for batch_input, batch_target in training_loader:
                optimizer.zero_grad()
                loss = criterion.forward(Net4.forward(batch_input), batch_target)
                if iters%20==0:
                    print("iter ",iters, " : ",loss)
                iters += 1
                grad_criterion = criterion.backward()
                #print("criterion: ",grad_criterion.norm())
                Net4.backward(grad_criterion)
                optimizer.step()
                
                epoch_loss += loss

            epochs_losses.append(epoch_loss / len(training_loader))
            # Validation
            
            
            validation_loss = 0
            psnr = 0
            for batch_input, batch_target in validation_loader:
                #if batch_input.shape[0]>1:
                    #ids = torch.randperm(batch_input.shape[0])
                    #sep = int(batch_input.shape[0]*0.5)
                    #sep_ids = ids[:sep]
                    #batch_input = batch_input#[sep_ids]
                    #batch_target = batch_target#[sep_ids]
                    
                batch_outputs = Net4.forward(batch_input)
                validation_loss += criterion.forward(batch_outputs, batch_target)#.data.item()
                #psnr_bis += -10*torch.log10( torch.mean((batch_outputs - batch_target)**2)+10**-8)
                    #psnr += 10 * torch.log10(1 / torch.nn.functional.mse_loss(batch_outputs, batch_target))
                psnr += 20*torch.log10(torch.tensor(1.0))-10*torch.log10(((batch_outputs - batch_target)**2).mean((1, 2, 3))).mean()
            validation_losses.append(validation_loss/len(validation_loader))
            validation_psnr.append(psnr / len(validation_loader))

            if Config.verbose:
                print(
                    f'Epoch: {epoch + 1}/{num_epochs} |train loss: {epochs_losses[-1]:.4f} |test loss: {validation_losses[-1]:.4f} |psnr(dB): {validation_psnr[-1]:.4f}')
            
            #if len(validation_losses) >=2:
                #if (validation_losses[-2]-validation_losses[-1] < 5e-4):
                    #optimizer.update_lr(0.5,"scale")

        if Config.verbose:
            print("Training finished. Best model saved.")
        elapsed_time = time.time()-start
        return epochs_losses, validation_losses, validation_psnr, elapsed_time



In [62]:
train(train_noisy_input, train_noisy_target,2)

Training started.
iter  20  :  tensor(0.0172)
iter  40  :  tensor(0.0177)
iter  60  :  tensor(0.0177)
iter  80  :  tensor(0.0186)
iter  100  :  tensor(0.0176)
iter  120  :  tensor(0.0185)
Epoch: 1/2 |train loss: 0.0183 |test loss: 0.0073 |psnr(dB): 22.0531
iter  140  :  tensor(0.0184)
iter  160  :  tensor(0.0184)
iter  180  :  tensor(0.0186)
iter  200  :  tensor(0.0191)
iter  220  :  tensor(0.0182)
iter  240  :  tensor(0.0180)
Epoch: 2/2 |train loss: 0.0183 |test loss: 0.0073 |psnr(dB): 22.0654
Training finished. Best model saved.


([tensor(0.0183), tensor(0.0183)],
 [tensor(0.0073), tensor(0.0073)],
 [tensor(22.0531), tensor(22.0654)],
 342.3880078792572)

In [9]:
params_Net4_22psnr = Net4.param()

In [12]:
params_Net4_22psnr[1]

(tensor([ 0.5689, -0.0387, -0.0348, -0.1787, -0.0307, -0.0192,  0.4054,  0.2363,
         -0.0655,  0.3182, -0.1200,  0.4364]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [22]:
x1 = [(1.,2.),(2.,3.)]
x2 = [(11.,22.),(22.,33.)]
x2.extend(x1)
for a,b in x2:
    print(a,b)

11.0 22.0
22.0 33.0
1.0 2.0
2.0 3.0


tensor([[[[ 0,  1,  4]]],


        [[[ 9, 16, 25]]],


        [[[36, 49, 64]]]])